In [1]:
import sys
import re
import requests
import json
from bs4 import BeautifulSoup

import urllib2
import socket

In [2]:
def load_credentials():
    lines = [line.rstrip('\n') for line in open('credentials.ini')]
    chars_to_strip = " \'\""
    for line in lines:
        if "client_id" in line:
            client_id = re.findall(r'[\"\']([^\"\']*)[\"\']', line)[0]
        if "client_secret" in line:
            client_secret = re.findall(r'[\"\']([^\"\']*)[\"\']', line)[0]
        #Currently only need access token to run, the other two perhaps for future implementation
        if "client_access_token" in line:
            client_access_token = re.findall(r'[\"\']([^\"\']*)[\"\']', line)[0]

    return client_id, client_secret, client_access_token

In [82]:
# Genius API credentials (available globally)
client_id, client_secret, client_access_token = load_credentials()
genius_url = "http://api.genius.com"
headers = {'Authorization': 'Bearer ' + client_access_token}

In [42]:
def search_genius(search_term):             
    querystring = "http://api.genius.com/search?q=" + urllib2.quote(search_term) + "&page=" + str(1)        
    # querystring = "http://api.genius.com/" + urllib2.quote(search_term) + "&page=" + str(1)       
    request = urllib2.Request(querystring)
    request.add_header("Authorization", "Bearer " + client_access_token)   
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)") #Must include user agent of some sort, otherwise 403 returned
    while True:
        try:
            response = urllib2.urlopen(request, timeout=4) #timeout set to 4 seconds; automatically retries if times out
            raw = response.read()
        except socket.timeout:
            print("Timeout raised and caught")
            continue
        break    

    return json.loads(raw)

In [79]:
def get_song_and_artist_ids(song_title, artist_name):  
    json_obj = search_genius(song_title + " " + artist_name)
    body = json_obj["response"]["hits"]
    body = body[0] # Just keep the first hit (for now)
    song_api   = body['result']['api_path']
    artist_api = body['result']['primary_artist']['api_path']
    
    return song_api, artist_api

**Download all song lyrics for a given artist**

In [167]:
# It's dumb, but you have to use an artist's song to get their artist ID
artist_name ='Chance the rapper'
song_title = 'Prom Night'
song_id, artist_id = get_song_and_artist_ids(song_title,artist_name)

# Okay, we have the artist API id, let's get a list of all of their songs on Genius
all_song_ids = []
# for page in range(1,10):
page = 1
while True:
    request = urllib2.Request(genius_url + artist_id + '/songs' + '?page=%d' % page)
    request.add_header("Authorization", "Bearer " + client_access_token)
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)")
    response = urllib2.urlopen(request, timeout=4)
    raw = response.read()
    json_obj = json.loads(raw)    
    [all_song_ids.append(song['api_path']) for song in json_obj['response']['songs']]

    num_songs = len(json_obj['response']['songs'])
    if json_obj['response']['next_page']==None:
        if page==1:
            print("No results for: " + search_term)
        break      
    print("Page {0} -- num songs {1}:".format(page, num_songs))    
    page += 1
    

print('Total songs found: {0}'.format(len(all_song_ids)))

Page 1 -- num songs 19:
Page 2 -- num songs 20:
Page 3 -- num songs 19:
Page 4 -- num songs 19:
Page 5 -- num songs 20:
Page 6 -- num songs 19:
Page 7 -- num songs 20:
Page 8 -- num songs 20:
Page 9 -- num songs 20:
Page 10 -- num songs 20:
Page 11 -- num songs 19:
Page 12 -- num songs 19:
Page 13 -- num songs 20:
Page 14 -- num songs 20:
Page 15 -- num songs 20:
Total songs found: 310


In [177]:
lyrics = lyrics_from_song_api_path(all_song_ids[2])
print(lyrics)


Yep, igh! igh! igh!
I got suspended, ooohh you got suspended
For chiefin' a hunnid blunts, 14,400 minutes
Fans all in the stands, they hands for Mr. Bennett
That racket over the net, ooohh give me my tennis
Shoes, give me a minute, ooohh I can't be tardy
My class is already started, they told my mom I'm retarded
But that 10Day done in one night, Honda from a Hyundai
No tassel in the spring, but after summer I'm alumni
They took my nigga one night, and I was standing right there
An inch away from Heaven, a million songs from right here
A step away from South by, a swing away from Cali
A hook away from verses, I'm a dad away from rapping
So at the school they arrested him
Back seat, squad car, nestled in
Shouts to the bitch nigga Heselton
Wit' his Big Show body ass wrestlin'
Finna see the 10Day Pestilence
So far, so good, so special and
No stars, good jobs, or excellents
And I'm still so fucking non-echelon
Ooohh you got suspended, ooohh you not gon' finish
Ooohh you look familiar, oooh

In [150]:
def lyrics_from_song_api_path(song_api_path):                   
    querystring = genius_url + song_api_path  
    # print(querystring)
    request = urllib2.Request(querystring)
    request.add_header("Authorization", "Bearer " + client_access_token)   
    request.add_header("User-Agent", "curl/7.9.8 (i686-pc-linux-gnu) libcurl 7.9.8 (OpenSSL 0.9.6b) (ipv6 enabled)") #Must include user agent of some sort, otherwise 403 returned
    while True:
        try:
            response = urllib2.urlopen(request, timeout=4) #timeout set to 4 seconds; automatically retries if times out
            raw = response.read()
        except socket.timeout:
            print("Timeout raised and caught")
            continue
        break

    json_obj = json.loads(raw)  

    # Get the URL to the song lyrics
    path = json_obj['response']['song']['path']
    page_url = "http://genius.com" + path
    page = requests.get(page_url)
    html = BeautifulSoup(page.text, "html.parser")  
    [h.extract() for h in html('script')]
    lyrics = str(html.find("div", class_="lyrics").get_text())
    lyrics = re.sub('\[.*\]','',lyrics) # Remove [Verse] and [Bridge] stuff
    lyrics = re.sub('\n{2}','',lyrics)  # Remove gaps between verses    
    return lyrics

**Older code (still useful)**

In [20]:
def write_lyrics_to_file(lyrics):   
    with open("Lyrics.txt", "a") as text_file:
        text_file.write('\n' + lyrics)

In [24]:
arguments = ['something','the beatles']
song_title = arguments[0].translate(None,"\'\"")
artist_name = arguments[1].translate(None,"\'\"")       
song_api_path = get_song_api(song_title, artist_name)   
lyrics = lyrics_from_song_api_path(song_api_path)   
print(lyrics)


Something in the way she moves
Attracts me like no other lover
Something in the way she woos me
I don't want to leave her now
You know I believe and how
Somewhere in her smile she knows
That I don't need no other lover
Something in her style that shows me
I don't want to leave her now
You know I believe and how
You're asking me will my love grow
I don't know, I don't know
You stick around and it may show
I don't know, I don't know
Something in the way she knows
And all I have to do is think of her
Something in the things she shows me
I don't want to leave her now
You know I believe and how
